### Uso de una gateway NAT para el acceso saliente a Internet desde subredes privadas

*Problema*
Usted tiene subredes públicas en su VPC que tienen una ruta a un gateway de Internet. Desea aprovechar esta configuración para proporcionar acceso a Internet sólo de salida para una instancia en subredes privadas.

*Solución*
Cree un gateway NAT en una de las subredes públicas. A continuación, cree una IP elástica y asóciela a la gateway NAT. En la tabla de rutas asociada a las subredes privadas, añada una ruta para el tráfico de salida a Internet que tenga como objetivo la gateway NAT.

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0205.png" width="700">

In [34]:
import boto3
import json
import time

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

In [2]:
# create VPC
vpc = ec2.create_vpc(CidrBlock='174.16.0.0/16')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBook"}])
vpc.wait_until_available()

In [3]:
# Crear routables
routetable_public = vpc.create_route_table()
routetable_public.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-RT-Public"}])

routetable_private1 = vpc.create_route_table()
routetable_private1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-RT-Private1"}])

routetable_private2 = vpc.create_route_table()
routetable_private2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-RT-Private2"}])

[ec2.Tag(resource_id='rtb-02c87c5edfac8c3dd', key='Name', value='AWSCookbook-RT-Private2')]

In [4]:
# Cree cuatro subredes:
subnet_1 = ec2.create_subnet(
    CidrBlock='174.16.0.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)
subnet_2 = ec2.create_subnet(
    CidrBlock='174.16.1.0/24',
    VpcId=vpc.id,
    AvailabilityZone='us-east-1b'
)
subnet_3 = ec2.create_subnet(
    CidrBlock='174.16.2.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)
subnet_4 = ec2.create_subnet(
    CidrBlock='174.16.3.0/24',
    VpcId=vpc.id,
    AvailabilityZone='us-east-1b'
)

In [5]:
subnet_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-Public-SN-1"}])
subnet_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-Public-SN-2"}])
subnet_3.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-Private-SN-1"}])
subnet_4.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-Private-SN-2"}])

[ec2.Tag(resource_id='subnet-08f58abdc493e8e5b', key='Name', value='AWSCookbook-Private-SN-2')]

In [6]:
# Crear un gateway de Internet (IGW):
internetgateway = ec2.create_internet_gateway(
    TagSpecifications=[{"ResourceType": "internet-gateway", 
    "Tags": [{
            "Key": "Name", 
            "Value": "AWSCookbook-IGW"}
            ]}
    ]
)

In [7]:
# Adjunte el gateway de Internet a la VPC existente:
vpc.attach_internet_gateway(InternetGatewayId=internetgateway.id)

{'ResponseMetadata': {'RequestId': '80d6ebd7-de3b-4a93-8b9d-67f5b23ac971',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '80d6ebd7-de3b-4a93-8b9d-67f5b23ac971',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '243',
   'date': 'Mon, 03 Oct 2022 18:44:32 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [8]:
# Cree una ruta de Internet para la tabla de enrutamiento pública:
routetable_public.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=internetgateway.id
)


ec2.Route(route_table_id='rtb-0eeeb4d26dcb06f9a', destination_cidr_block='0.0.0.0/0')

In [9]:
# Asociar la tabla de rutas con las subredes:
routetable_public.associate_with_subnet(SubnetId = subnet_1.id)
routetable_public.associate_with_subnet(SubnetId = subnet_2.id)
routetable_private1.associate_with_subnet(SubnetId = subnet_3.id)
routetable_private2.associate_with_subnet(SubnetId = subnet_4.id)


ec2.RouteTableAssociation(id='rtbassoc-034a88ab00fc350c2')

In [10]:
# Create a security group
security_group = vpc.create_security_group(
    GroupName='AWSCookbook-SG-SSH',
    Description='Allow SSH access',
    VpcId=vpc.id
)
security_group.authorize_ingress(
    CidrIp=vpc.cidr_block,
    IpProtocol='tcp',
    FromPort=22,
    ToPort=22
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-0e88b71f39a09fa96',
   'GroupId': 'sg-07072c1d8d846ee1d',
   'GroupOwnerId': '655734544982',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'CidrIpv4': '174.16.0.0/16'}],
 'ResponseMetadata': {'RequestId': '8f09afaf-b637-4deb-8eb0-e26089448baf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8f09afaf-b637-4deb-8eb0-e26089448baf',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '723',
   'date': 'Mon, 03 Oct 2022 18:44:35 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [12]:
# Create a key pair
key_name = 'AWSCookbook-KeyPair-1'
keypair = ec2.create_key_pair(KeyName=key_name)

In [13]:
# Cear una instancia EC2
instance_1 = ec2.create_instances(
    ImageId='ami-0c2b8ca1dad447f8a',
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName=key_name,
    SubnetId=subnet_3.id,
    SecurityGroupIds=[security_group.id],
    TagSpecifications=[{"ResourceType": "instance",
                        "Tags": [{"Key": "Name", "Value": "AWSCookbook-Instance-Private-1"}]}]

)

In [94]:
instance1_id = instance_1[0].id

In [14]:
# Crear una Elastic IP address
elastic_ip = ec2_client.allocate_address(Domain='vpc')

In [36]:
# Crear un NAT gateway
natgateway = ec2_client.create_nat_gateway(
    AllocationId=elastic_ip['AllocationId'],
    SubnetId=subnet_1.id
)


In [52]:
try:
    natgateway_id = natgateway['NatGateways'][0]['NatGatewayId']
except:
    natgateway_id = natgateway['NatGateway']['NatGatewayId']

El estado tardará unos instantes en estar disponible:

In [53]:
if natgateway['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('NAT Gateway created successfully')
    print('Waiting for NAT Gateway to be available...')
    while True:
        time.sleep(10)
        natgateway_status = ec2_client.describe_nat_gateways(NatGatewayIds=[natgateway_id])
        if natgateway_status['NatGateways'][0]['State'] == 'available':
            print('NAT Gateway is available')
            break

NAT Gateway created successfully
Waiting for NAT Gateway to be available...
NAT Gateway is available


Añade una ruta por defecto para 0.0.0.0/0 con destino a la gateway NAT a las dos tablas de rutas del nivel privado. Esta ruta por defecto envía todo el tráfico que no coincide con una ruta más específica al destino especificado:

In [55]:
routetable_private1.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    NatGatewayId=natgateway_id
)
routetable_private2.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    NatGatewayId=natgateway_id
)

ec2.Route(route_table_id='rtb-02c87c5edfac8c3dd', destination_cidr_block='0.0.0.0/0')

#### Conéctese a la instancia 1 de EC2 mediante el gestor de sesiones SSM

In [56]:
iam = boto3.client('iam')
ssm = boto3.client('ssm')

In [57]:
# Create a role
role_name = 'Cookbook-SSM-Role'
policy_document = {
  "Version": "2012-10-17",
  "Statement": [
  {
    "Effect": "Allow",
    "Principal": {
      "Service": "ec2.amazonaws.com"
    },
    "Action": "sts:AssumeRole"
  }
  ]
}

ssm_role = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(policy_document)
)

In [58]:
# Attach the policy to the role
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'
)

{'ResponseMetadata': {'RequestId': '13bfa2a6-13ae-420d-84dd-c6e1ba04bbd0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '13bfa2a6-13ae-420d-84dd-c6e1ba04bbd0',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 03 Oct 2022 19:04:47 GMT'},
  'RetryAttempts': 0}}

In [59]:
# Create an instance profile
instance_profile_name = 'Cookbook-SSM-Profile'
instance_profile = iam.create_instance_profile(
    InstanceProfileName=instance_profile_name
)


In [60]:
# Add the role that you created to the instance profile:
iam.add_role_to_instance_profile(
    InstanceProfileName=instance_profile_name,
    RoleName=role_name
)

{'ResponseMetadata': {'RequestId': '7e70f148-691f-47e0-b1f9-15d131eb995f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7e70f148-691f-47e0-b1f9-15d131eb995f',
   'content-type': 'text/xml',
   'content-length': '228',
   'date': 'Mon, 03 Oct 2022 19:04:53 GMT'},
  'RetryAttempts': 0}}

In [61]:
# Consulte en SSM el último ID de AMI de Amazon Linux 2 disponible en su región y guárdelo como variable de entorno:
ssm_response = ssm.get_parameter(
    Name='/aws/service/ami-amazon-linux-latest/amzn2-ami-hvm-x86_64-gp2'
)

In [62]:
# Create a key pair
key_name = 'AWSCookbook-KeyPair-SSM-Instance'
keypair = ec2.create_key_pair(KeyName=key_name)

In [63]:
instance = ec2.create_instances(
    ImageId=ssm_response['Parameter']['Value'],
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName=key_name,
    SubnetId=subnet_3.id,
    IamInstanceProfile={
        'Name': 'Cookbook-SSM-Profile'
    },
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'Cookbook-SSM-Instance'
                },
            ]
        },
    ]
)

In [88]:
# Esperar a que la instancia esté disponible
instance[0].wait_until_running()

In [84]:
instance_id = ssm.describe_instance_information()['InstanceInformationList'][0]['InstanceId']
print("En la terminal ejecute el siguiente comando:")
print("   aws ssm start-session --target " + instance_id)

En la terminal ejecute el siguiente comando:
   aws ssm start-session --target i-07447844b7aa3152d


Prueba el acceso a Internet con un ping:

In [ ]:
ping -c 4 homestarrunner.com

#### Discusión
Esta arquitectura le da un nivel de subred que permite el acceso saliente pero no permite el acceso entrante directo de Internet a los recursos dentro de ella. Una forma de permitir el acceso entrante de los recursos de Internet a los servicios que se ejecutan en los recursos de las subredes privadas es utilizar un equilibrador de carga en las subredes públicas. Veremos más sobre este tipo de configuración en la Receta 2.7.

La EIP asociada a su gateway NAT se convierte en la dirección IP externa para toda la comunicación que pasa por ella. Por ejemplo, si un proveedor necesitara añadir su IP a una lista de permitidos, el EIP de la pasarela NAT sería la dirección IP "de origen" proporcionada al proveedor. Tu EIP seguirá siendo el mismo mientras lo mantengas provisionado dentro de tu cuenta.

TIP:
* Si ha creado una VPC con capacidad para IPv6, también puede crear una gateway de Internet de sólo salida para permitir el acceso a Internet saliente para subredes privadas, como se explica en un artículo de AWS.

Este gateway NAT se aprovisionó dentro de un AZ en su VPC. Si bien esta es una forma rentable de lograr el acceso a Internet saliente para sus subredes privadas, para las aplicaciones de producción y de misión crítica, debería considerar el aprovisionamiento de gateways NAT en cada AZ para proporcionar resiliencia y reducir la cantidad de tráfico entre zonas. Esto también requeriría la creación de tablas de rutas para cada una de sus subredes privadas para que pueda dirigir el tráfico 0.0.0/0 a la gateway NAT en la AZ de esa subred en particular. Consulte el reto de esta receta.

**NOTA:** Si tiene requisitos personalizados o desea un control más granular del enrutamiento saliente para su implementación de NAT, puede crear una instancia de NAT. Para una comparación de las gateways NAT y las instancias NAT, consulte este documento de soporte.

<br>
<br>

-------------

**Desafío**

Cree un segundo gateway NAT en la subred pública en AZ2. A continuación, modifique la ruta por defecto en la tabla de rutas asociada a la subred privada en AZ2. Cambie el destino a la gateway NAT recién creada.

#### Limpieza

In [96]:
# delete instances
ec2_client.terminate_instances(InstanceIds=[instance_id])
ec2_client.terminate_instances(InstanceIds=[instance1_id])
# delete security group
ec2_client.delete_security_group(GroupId=security_group.id)
# delete the NAT gateway
ec2_client.delete_nat_gateway(NatGatewayId=natgateway_id)
# delete internet gateway
ec2_client.detach_internet_gateway(InternetGatewayId=internetgateway.id, VpcId=vpc.id)
ec2_client.delete_internet_gateway(InternetGatewayId=internetgateway.id)
# delete subnet
ec2_client.delete_subnet(SubnetId=subnet_1.id)
ec2_client.delete_subnet(SubnetId=subnet_2.id)
ec2_client.delete_subnet(SubnetId=subnet_3.id)
ec2_client.delete_subnet(SubnetId=subnet_4.id)
# delete route table
ec2_client.delete_route_table(RouteTableId=routetable_public.id)
ec2_client.delete_route_table(RouteTableId=routetable_private1.id)
ec2_client.delete_route_table(RouteTableId=routetable_private2.id)
# delete a elastic IP
ec2_client.release_address(AllocationId=elastic_ip['AllocationId'])
# delete VPC
ec2_client.delete_vpc(VpcId=vpc.id)
# remove roles from instance profile
iam.remove_role_from_instance_profile(
    InstanceProfileName=instance_profile_name,
    RoleName=ssm_role['Role']['RoleName']
)
# delete instance profile
iam.delete_instance_profile(InstanceProfileName=instance_profile_name)

{'TerminatingInstances': [{'CurrentState': {'Code': 48, 'Name': 'terminated'},
   'InstanceId': 'i-0e30ec0f49b19f419',
   'PreviousState': {'Code': 80, 'Name': 'stopped'}}],
 'ResponseMetadata': {'RequestId': 'db309906-399a-4816-83a3-53a8c7b85d1a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'db309906-399a-4816-83a3-53a8c7b85d1a',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Mon, 03 Oct 2022 19:25:41 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [ ]:
# delete role
iam.delete_role(RoleName=ssm_role['Role']['RoleName'])